In [61]:
logger_name = "lstnet"
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import seaborn as sns
# Path appended in order to import from util
import sys
sys.path.append('..')
from util.model_util import LoadModel, SaveModel, SaveResults, SaveHistory
from util.Msglog import LogInit

from datetime import datetime
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import Input, GRU, Conv2D, Dropout, Flatten, Dense, Reshape, Concatenate, Add
from tensorflow.keras.optimizers import SGD, RMSprop, Adam

from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard


from LSTNet.lstnet_util import GetArguments, LSTNetInit
from LSTNet.lstnet_datautil import DataUtil
from LSTNet.lstnet_model import PreSkipTrans, PostSkipTrans, PreARTrans, PostARTrans, LSTNetModel, ModelCompile
from LSTNet.lstnet_plot import AutoCorrelationPlot, PlotHistory, PlotPrediction


In [62]:
def GetArguments():
    # Creating the argument parser
    import argparse
    parser = argparse.ArgumentParser(description='LSTNet Model')
    
    #parser.add_argument('--data', type=str, required=True, help='testing\LSTNet implementation\portfolio_data.csv')
    parser.add_argument('--window', type=int, default=24*7, help='Window size. Default=24*7')
    parser.add_argument('--horizon', type=int, default=12, help='Horizon width. Default=12')
    parser.add_argument('--CNNFilters', type=int, default=100, help='Number of CNN layer filters. Default=100. If set to 0, the CNN layer will be omitted')
    parser.add_argument('--CNNKernel', type=int, default=6, help='Size of the CNN filters. Default=6. If set to 0, the CNN layer will be omitted')
    parser.add_argument('--GRUUnits', type=int, default=100, help='Number of GRU hidden units. Default=100')
    parser.add_argument('--SkipGRUUnits', type=int, default=5, help='Number of hidden units in the Skip GRU layer. Default=5')
    parser.add_argument('--skip', type=int, default=24,
                        help='Size of the window to skip in the Skip GRU layer. Default=24. If set to 0, the SkipGRU layer will be omitted')
    parser.add_argument('--dropout', type=float, default=0.2, help='Dropout to be applied to layers. 0 means no dropout. Default=0.2')    
    parser.add_argument('--normalize', type=int, default=2,
                        help='0 = do not normalise, 1 = use same normalisation for all timeseries, 2 = normalise each timeseries independently. Default=2')
    parser.add_argument('--highway', type=int, default=24, help='The window size of the highway component. Default=24. If set to 0, the AR layer will be omitted')
    parser.add_argument('--lr', type=float, default=0.001, help='Learning rate. Default=0.001')
    parser.add_argument('--batchsize', type=int, default=128, help='Training batchsize. Default=128')
    parser.add_argument('--epochs', type=int, default=100, help='Number of epochs to run for training. Default=100')
    parser.add_argument('--initializer', type=str, default="glorot_uniform", help='Weights initialiser to use. Default=glorot_uniform')
    parser.add_argument('--loss', type=str, default="mean_absolute_error", help='Loss function to use. Default=mean_absolute_error')
    parser.add_argument('--optimizer', type=str, default="Adam", help='Optimisation function to use. Default=Adam')
    parser.add_argument('--trainpercent', type=float, default=0.6, help='Percentage of data to be used for training. Default=0.6')
    parser.add_argument('--validpercent', type=float, default=0.2, help='Percentage of data to be used for validation. Default=0.2')
    parser.add_argument('--save', type=str, default=None, help='Filename initial to save the model and the results in. Default=None')
    parser.add_argument('--load', type=str, default=None, help='Filename initial of the saved model to be loaded (model and weights). Default=None')
    parser.add_argument('--tensorboard', type=str, default=None,
                        help='Location of the tensorboard folder. If not set, tensorboard will not be launched. Default=None i.e. no tensorboard callback')
    parser.add_argument('--predict', type=str, choices=['trainingdata', 'validationdata', 'testingdata', 'all', None], default=None,
                        help='Predict timesseries. Default None')
    parser.add_argument('--series-to-plot', type=str, default='0', help='Series to plot. Default 0 (i.e. plot the first timeseries)')
    parser.add_argument('--autocorrelation', type=str, default=None,
                        help='Plot an autocorrelation of the input data. Format --autocorrelation=i,j,k which means to plot an autocorrelation of timeseries i from timeslot j to timeslot k')
    parser.add_argument('--save-plot', type=str, default=None, help='Filename initial to save the plots to in PNG format. Default=None')

    parser.add_argument('--no-train', action='store_true', help='Do not train model.')
    parser.add_argument('--no-validation', action='store_true',
                        help='Do not validate model. When not set and no-train is not set, data will be validated while training')
    parser.add_argument('--test', action='store_true', help='Test model.')
    parser.add_argument('--no-saveresults', action='store_true', help='Do not save training / validation results.')
    parser.add_argument('--savehistory', action='store_true', help='Save training / validation history.')
    parser.add_argument('--plot', action='store_true', help='Generate plots.')

    parser.add_argument('--no-log', action='store_true', help='Do not create log files. Only error and critical messages will appear on the console.')
    parser.add_argument('--debuglevel', type=int, choices=[10, 20, 30, 40, 50], default=20, help='Logging debug level. Default 20 (INFO)')
    parser.add_argument('--logfilename', type=str, default="log/lstnet", help="Filename where logging will be written. Default: log/lstnet")

    args, unknown = parser.parse_known_args()
    return args

In [63]:
args=GetArguments()
args_dict=vars(args)
args_dict["data"]='testing\LSTNet implementation\portfolio_data.csv'

In [64]:
args_dict

{'window': 168,
 'horizon': 12,
 'CNNFilters': 100,
 'CNNKernel': 6,
 'GRUUnits': 100,
 'SkipGRUUnits': 5,
 'skip': 24,
 'dropout': 0.2,
 'normalize': 2,
 'highway': 24,
 'lr': 0.001,
 'batchsize': 128,
 'epochs': 100,
 'initializer': 'glorot_uniform',
 'loss': 'mean_absolute_error',
 'optimizer': 'Adam',
 'trainpercent': 0.6,
 'validpercent': 0.2,
 'save': None,
 'load': None,
 'tensorboard': None,
 'predict': None,
 'series_to_plot': '0',
 'autocorrelation': None,
 'save_plot': None,
 'no_train': False,
 'no_validation': False,
 'test': False,
 'no_saveresults': False,
 'savehistory': False,
 'plot': False,
 'no_log': False,
 'debuglevel': 20,
 'logfilename': 'log/lstnet',
 'data': 'testing\\LSTNet implementation\\portfolio_data.csv'}

In [65]:
lstnet_init = LSTNetInit(args=args_dict, args_is_dictionary=True)

# Access the attributes of the object as needed
print(lstnet_init.data)
print(lstnet_init.window)
print(lstnet_init.horizon)

testing\LSTNet implementation\portfolio_data.csv
168
12


In [87]:
lstnet_instance=LSTNetModel(init=lstnet_init,input_shape=(1,1520,5))

In [72]:
from LSTNet.main import train
train

NameError: name 'X' is not defined

In [105]:
np.savetxt(fname='portfolio_data.txt',X=np.array(arr),delimiter=',')

In [80]:
df=df.drop('Date',axis=1)
df.r

,Date,AMZN,DPZ,BTC,NFLX
0,5/1/2013,248.229996,51.190983,106.250000,30.415714
1,5/2/2013,252.550003,51.987320,98.099998,30.641428
2,5/3/2013,258.049988,52.446388,112.900002,30.492857
3,5/6/2013,255.720001,53.205257,109.599998,30.098572
4,5/7/2013,257.730011,54.151505,113.199997,29.464285
...,...,...,...,...,...
1515,5/8/2019,1917.770020,283.149994,6171.959961,364.369995
1516,5/9/2019,1899.869995,282.160004,6358.290039,362.750000
1517,5/10/2019,1889.979980,278.369995,7191.359863,361.040009
1518,5/13/2019,1822.680054,273.880005,7980.129883,345.260010


In [99]:
arr=np.array(df)[:,1:]
pd.DataFrame(arr).to_csv('inp_data.csv')

In [100]:
pd.read_csv('inp_data.csv')

,Unnamed: 0,0,1,2,3
0,0,248.229996,51.190983,106.250000,30.415714
1,1,252.550003,51.987320,98.099998,30.641428
2,2,258.049988,52.446388,112.900002,30.492857
3,3,255.720001,53.205257,109.599998,30.098572
4,4,257.730011,54.151505,113.199997,29.464285
...,...,...,...,...,...
1515,1515,1917.770020,283.149994,6171.959961,364.369995
1516,1516,1899.869995,282.160004,6358.290039,362.750000
1517,1517,1889.979980,278.369995,7191.359863,361.040009
1518,1518,1822.680054,273.880005,7980.129883,345.260010


In [101]:
dataloader=DataUtil(filename='inp_data.csv',horizon=args_dict["horizon"],window=args_dict["window"],train=0.75,valid=0.25)

ValueError: could not convert string to float: ''

In [67]:
history=ModelCompile(model=lstnet_instance,init=lstnet_init)

c:\Users\ved14\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [55]:
history

In [50]:
df=pd.read_csv('portfolio_data.csv')
df

,Date,AMZN,DPZ,BTC,NFLX
0,5/1/2013,248.229996,51.190983,106.250000,30.415714
1,5/2/2013,252.550003,51.987320,98.099998,30.641428
2,5/3/2013,258.049988,52.446388,112.900002,30.492857
3,5/6/2013,255.720001,53.205257,109.599998,30.098572
4,5/7/2013,257.730011,54.151505,113.199997,29.464285
...,...,...,...,...,...
1515,5/8/2019,1917.770020,283.149994,6171.959961,364.369995
1516,5/9/2019,1899.869995,282.160004,6358.290039,362.750000
1517,5/10/2019,1889.979980,278.369995,7191.359863,361.040009
1518,5/13/2019,1822.680054,273.880005,7980.129883,345.260010


In [19]:
args

{'data': 'testing\\LSTNet implementation\\portfolio_data.csv',
 'window': 14,
 'horizon': 7,
 'CNNFilters': 3,
 '--ip': '127.0.0.1',
 '--stdin': '9011',
 '--control': '9009',
 '--hb': '9008',
 '--Session.signature_scheme': '"hmac-sha256"',
 '--Session.key': 'b"4d9e432c-8593-49f8-bfa2-799a3f1fa63d"',
 '--shell': '9010',
 '--transport': '"tcp"',
 '--iopub': '9012',
 '--f': 'c:\\Users\\ved14\\AppData\\Roaming\\jupyter\\runtime\\kernel-v2-8816Xiv2TvLnQxKp.json'}

In [5]:
data_initilizer=DataUtil()

TypeError: __init__() missing 5 required positional arguments: 'filename', 'train', 'valid', 'horizon', and 'window'

In [6]:
df=pd.read_csv('portfolio_data.csv')
data=np.array(df)
from LSTNet.main import train
input_shape=[]
model=LSTNetModel(data=np.array(df))

ModuleNotFoundError: No module named 'lstnet_util'